In [5]:
import requests
import json
from typing import List, Dict, Any, Tuple
import time

GAMMA_BASE = "https://gamma-api.polymarket.com"
CLOB_BASE  = "https://clob.polymarket.com"


def fetch_active_events(limit: int = 200, offset: int = 0):
    params = {
        "closed": "false",
        "limit": str(limit),
        "offset": str(offset),
        "order": "volume",
        "ascending": "false",
    }
    r = requests.get(f"{GAMMA_BASE}/events", params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    if isinstance(data, dict) and "results" in data:
        return data["results"]
    if isinstance(data, list):
        return data
    return []


def parse_clob_ids(market: Dict[str, Any]) -> List[str]:
    raw = market.get("clobTokenIds")
    if not raw:
        return []
    if isinstance(raw, list):
        return [str(x) for x in raw]
    try:
        if raw.startswith("["):
            return json.loads(raw)
        return [x.strip() for x in raw.split(",") if x.strip()]
    except:
        return []


def fetch_best_ask(token_id: str) -> float | None:
    try:
        r = requests.get(
            f"{CLOB_BASE}/book",
            params={"token_id": token_id},
            timeout=3
        )
        if r.status_code != 200:
            return None
        data = r.json()
        asks = data.get("asks", [])
        if not asks:
            return None
        return float(asks[0]["price"])
    except:
        return None


def find_bundle_arbs(min_edge: float = 0.02):
    events = fetch_active_events(limit=200)
    print(f"Fetched {len(events)} events.\n")

    arbs = []
    total_books = 0
    max_books = 180  # still fast

    for evt in events:
        markets = evt.get("markets", [])

        # only multi-outcome (>2 markets) events matter
        if len(markets) < 3:
            continue

        bundle = []

        for m in markets:
            ids = parse_clob_ids(m)
            if not ids:
                continue

            yes_token = ids[0]

            total_books += 1
            if total_books % 20 == 0:
                print(f"Checked {total_books} orderbooks...")

            if total_books > max_books:
                print("Reached max book limit. Stopping early.")
                return arbs

            ask = fetch_best_ask(yes_token)
            if ask is None:
                continue

            bundle.append((m.get("id",""), m.get("question",""), ask))

        if len(bundle) < 3:
            continue

        total_cost = sum(x[2] for x in bundle)

        if total_cost < 1 - min_edge:
            arbs.append({
                "event_id": evt.get("id"),
                "event_title": evt.get("title") or evt.get("question"),
                "total_cost": total_cost,
                "edge": 1 - total_cost,
                "markets": [
                    {"market_id": mid, "question": q, "best_ask": p}
                    for mid, q, p in bundle
                ],
            })

    return arbs


if __name__ == "__main__":
    start = time.time()
    arbs = find_bundle_arbs(min_edge=0.01)
    elapsed = time.time() - start

    print(f"\nScan finished in {elapsed:.2f} seconds.")
    print(f"Found {len(arbs)} arbitrage bundles.\n")

    for opp in arbs:
        print("=" * 70)
        print(f"EVENT: {opp['event_title']}")
        print(f"Total cost: {opp['total_cost']:.4f}")
        print(f"Edge: {100*opp['edge']:.2f}%")
        print("-----------------------------------------")
        for m in opp["markets"]:
            print(f"  {m['question']} — {m['best_ask']:.4f}")


Fetched 200 events.

Checked 20 orderbooks...
Checked 40 orderbooks...
Checked 60 orderbooks...
Checked 80 orderbooks...
Checked 100 orderbooks...
Checked 120 orderbooks...
Checked 140 orderbooks...
Checked 160 orderbooks...
Checked 180 orderbooks...
Reached max book limit. Stopping early.

Scan finished in 23.87 seconds.
Found 0 arbitrage bundles.



In [7]:
import requests
import json
import time

GAMMA = "https://gamma-api.polymarket.com"
CLOB  = "https://clob.polymarket.com"


def fetch_events(offset: int = 0, limit: int = 50):
    params = {
        "closed": "false",
        "limit": limit,
        "offset": offset,
        "order": "liquidity",      # VALID ORDER FIELD
        "ascending": "false",
    }
    r = requests.get(f"{GAMMA}/events", params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    if isinstance(data, dict) and "results" in data:
        return data["results"]
    return data


def parse_ids(market):
    raw = market.get("clobTokenIds")
    if not raw:
        return []
    if isinstance(raw, list):
        return [str(x) for x in raw]
    try:
        if raw.startswith("["):
            return json.loads(raw)
        return [x.strip() for x in raw.split(",") if x.strip()]
    except:
        return []


def best_ask(token_id):
    try:
        r = requests.get(f"{CLOB}/book", params={"token_id": token_id}, timeout=2)
        if r.status_code != 200:
            return None
        asks = r.json().get("asks", [])
        if not asks:
            return None
        return float(asks[0]["price"])
    except:
        return None


def scan_arbs(pages=12, limit=50):
    arbs = []
    total_books = 0
    max_books = 250  # hard cap to keep fast

    for page in range(pages):
        offset = page * limit
        print(f"Page {page}, offset {offset}")

        events = fetch_events(offset=offset, limit=limit)
        print(f"  → got {len(events)} events")

        for evt in events:

            markets = evt.get("markets", [])
            if len(markets) < 3:
                continue  # skip binary

            prices = []
            for m in markets:
                ids = parse_ids(m)
                if not ids:
                    continue

                total_books += 1
                if total_books % 20 == 0:
                    print(f"Checked {total_books} books...")

                if total_books > max_books:
                    print("Hit max book limit, stopping early.")
                    return arbs

                ask = best_ask(ids[0])
                if ask is None:
                    continue

                prices.append((m.get("question",""), ask))

                # limit per event for speed
                if len(prices) >= 6:
                    break

            if len(prices) < 3:
                continue

            total = sum(p[1] for p in prices)

            if total < 1:
                arbs.append({
                    "event": evt.get("title") or evt.get("question"),
                    "total_cost": total,
                    "edge": round(1 - total, 4),
                    "markets": prices,
                })

    return arbs


if __name__ == "__main__":
    start = time.time()
    arbs = scan_arbs(pages=12)
    print(f"\nFinished in {time.time() - start:.2f}s")
    print(f"Found {len(arbs)} arbs.\n")

    for a in arbs:
        print("="*60)
        print(a["event"])
        print(f"Total cost: {a['total_cost']:.4f}, edge: {a['edge']*100:.2f}%")
        for q,p in a["markets"]:
            print(f"   {p:.4f} – {q}")


Page 0, offset 0
  → got 50 events
Checked 20 books...
Checked 40 books...
Checked 60 books...
Checked 80 books...
Checked 100 books...
Checked 120 books...
Checked 140 books...
Checked 160 books...
Checked 180 books...
Checked 200 books...
Checked 220 books...
Checked 240 books...
Hit max book limit, stopping early.

Finished in 35.37s
Found 0 arbs.

